## Задача 2.

Здесь будут реализованы методы численного интегрирования. Данную задачу можно посмотреть [здесь](https://drive.google.com/file/d/1q3XLkKtb8e6DBn_5X1EbbFHuSAXN0j6k/view), она находится под номером 2.

In [1]:
import numpy as np

Определим сразу все возможные функции для нашей задачи.

In [2]:
def f_1(x):
    return np.sin(x)


def f_2(x):
    return (x - 1) / (np.log(x))


def f_3(x):
    return np.abs(x)

Заведем функцию `sympson`, она будет отвечать за приближенное вычисление интеграла от функции $f(x)$ на отрезке $[x_i, x_{i+1}]$.

In [3]:
def sympson(x_0, x_2, f):
    return (f(x_0) + 4 * f((x_0 + x_2) / 2) + f(x_2)) * (x_2 - x_0) / 6

Заведем функцию gauss, она будет отвечать за приближенное вычисление интеграла от функции $f(x)$ на отрезке $[x_i, x_{i+1}]$.

Будем делать следующим образом: заметим, что (смотреть [здесь](https://scask.ru/i_book_clm.php?id=101))
$$ f(t) = \frac{a+b}{2} + \frac{b-a}{2}t $$
биективное отображение отрезка $[-1, 1]$ в отрезок $[a, b]$. Стало быть, достаточно знать узлы и веса на отрезке $[-1, 1]$. В нашей задаче 3 узла. Можно либо решать СЛАУ, либо найти значения вот [здесь](http://aco.ifmo.ru/el_books/numerical_methods/lectures/glava2_3.html). Заведем два `np.array` под них:

In [4]:
nodes = np.array([-0.7745967, 0, 0.7745967]) # узлы
weights = np.array([0.5555556, 0.8888889, 0.5555556]) # веса

In [5]:
def gauss(x_0, x_1, f):
    new_nodes = (x_0 + x_1) / 2 + (x_1 - x_0) / 2 * nodes
    return f(new_nodes.T) @ weights

Теперь заведем функцию, которая будет по составной квадратурной формуле считать интеграл. На вход подается метод, пределы интегрирования, количество интервалов и функция. В методе Гаусса в конце нужно домножить на $(b-a)/2n$, так как делали замену переменных в интеграле на каждом промежутке и всегда вылетал якобиан $(b-a)/2n$!

In [6]:
def NI(name='sympson', a=0, b=np.pi, n=100, function=f_1): # По дефолту считаем интеграл от sin(x) на [0, pi] методом Симпсона
    total = 0
    grid = np.linspace(start=a, stop=b, num=(n + 1)) # Равномерная сетка
    if name == 'sympson':
        for i in range(n):
            total += sympson(grid[i], grid[i + 1], function)
        return total
    elif name == 'gauss':
        for i in range(n):
            total += gauss(grid[i], grid[i + 1], function)
        return (b - a) / (2 * n) * total

# Метод Симпсона

Можно смотреть [здесь](https://ru.wikipedia.org/wiki/Формула_Симпсона).

## Compute

**Вычислим все три интеграла на равномерной сетке со 100 интервалами методом Симпсона.**

In [7]:
num_1 = NI('sympson', 0, np.pi, 100)

In [8]:
print(f'Для первой функции: {num_1}')

Для первой функции: 2.0000000006764718


In [9]:
num_2 = NI('sympson', 0, 10, 100, f_2) # Здесь пытались считать f(0), а оно не определено, так как возникает деление на ноль

C:\Users\csc20\AppData\Local\Temp\ipykernel_10700\1762032353.py:6: RuntimeWarning: divide by zero encountered in log
  return (x - 1) / (np.log(x))
C:\Users\csc20\AppData\Local\Temp\ipykernel_10700\1762032353.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return (x - 1) / (np.log(x))


In [10]:
print(f'Для второй функции: {num_2}')

Для второй функции: nan


In [11]:
num_3 = NI('sympson', -1, 2, 100, f_3)

In [12]:
print(f'Для третьей функции: {num_3}')

Для третьей функции: 2.4999999999999996


Очевидно, вторая функция не определена в 0 и 1. Если попробовать доопределить 0 и 1 в точках 0, 1 по непрерывности, то уже результат будет.

In [13]:
def f_2_1(x):
    if x in [0, 1]:
        return x
    return (x - 1) / np.log(x)

In [14]:
num_2_1 = NI('sympson', 0, 10, 100, f_2_1)

In [15]:
print(f'Для второй функции (измененной): {num_2_1}')

Для второй функции (измененной): 23.957634906766412


**Теперь воспользуемся апостериорной оценкой Рунге и предскажем порядок.**

## Апостериорная оценка Рунге

### Case 1

In [16]:
f_1_200 = NI('sympson', 0, np.pi, 200)
f_1_400 = NI('sympson', 0, np.pi, 400)
f_1_800 = NI('sympson', 0, np.pi, 800)
print(f'Для 200 интервалов: {f_1_200}\nДля 400 интервалов: {f_1_400}\nДля 800 интервалов: {f_1_800}')

Для 200 интервалов: 2.0000000000422786
Для 400 интервалов: 2.000000000002641
Для 800 интервалов: 2.000000000000163


Апостериорная оценка Рунге. Определение порядка точности квадратурной формулы при неизвестном точном значение интеграла
$$ \frac{I_{h/2} - I_{h}}{I_{h/4} - I_{h/2}} \thickapprox 2^p, $$
где $I_h$ - приближенное значение интеграла на равномерной сетке с длинной интервала $h=(b-a)/n$, а $p$ - порядок точности квадратурной формулы. У метода Симпсона $p=4$.

In [17]:
np.log2((f_1_400 - f_1_200)/(f_1_800 - f_1_400))

3.999612126616594

Как мы видим, предсказанный порядок примерно 4, что соответствует теоретической оценке.

Аналогично сделаем для двух оставшихся функций.

### Case 2

Мы уже говорили о том, что у функции есть проблемы в двух точках, поэтому нет смысла считать в лоб, поэтому сделаем, как уже делали до этого, то есть используем новую функцию (с доопределенными значениями в нуле и единице).

In [18]:
f_2_1_200 = NI('sympson', 0, 10, 200, f_2_1)
f_2_1_400 = NI('sympson', 0, 10, 400, f_2_1)
f_2_1_800 = NI('sympson', 0, 10, 800, f_2_1)
print(f'Для 200 интервалов: {f_2_1_200}\nДля 400 интервалов: {f_2_1_400}\nДля 800 интервалов: {f_2_1_800}')

Для 200 интервалов: 23.959238139005326
Для 400 интервалов: 23.959951648167202
Для 800 интервалов: 23.96027253587817


В итоге получим некоторый порядок:

In [19]:
np.log2((f_2_1_400 - f_2_1_200)/(f_2_1_800 - f_2_1_400))

1.1528634150716075

Получился порядок, который вообще не сходится с теоретическим. Дело в том, что в апостериорной оценке Рунге мы пользовались предположением
$$ I - I_h \thickapprox Ch^p(b-a), $$
для некоторой константы $C$. Но проблема в том, что у функции производные в нуле и единице улетают на бесконечность, а значит ни о какой формуле Тейлора с остаточным членом в форме Лагранжа нет и речи.

### Case 3

In [20]:
f_3_200 = NI('sympson', -1, 2, 200, f_3)
f_3_400 = NI('sympson', -1, 2, 400, f_3)
f_3_800 = NI('sympson', -1, 2, 800, f_3)
print(f'Для 200 интервалов: {f_3_200}\nДля 400 интервалов: {f_3_400}\nДля 800 интервалов: {f_3_800}')

Для 200 интервалов: 2.5
Для 400 интервалов: 2.5
Для 800 интервалов: 2.4999999999999973


In [21]:
np.log2((f_3_400 - f_3_200)/(f_3_800 - f_3_400))

C:\Users\csc20\AppData\Local\Temp\ipykernel_10700\2072337812.py:1: RuntimeWarning: divide by zero encountered in log2
  np.log2((f_3_400 - f_3_200)/(f_3_800 - f_3_400))


-inf

Здесь же опять у нас возникают проблемы, ведь функция не дифф. в нуле.

## Априорная оценка

Априорная оценка для достаточно гладкой функции $f(x)$ на равномерной сетке с шагом $h=(b-a)/n$ при вычисление приближенного значения $I_h$ интеграла $I$ имеет вид
$$ \mid I-I_{h}\mid \leq \frac{M_4(b-a)h^4}{2880}, $$
где $M_4$ -- максимум 4 производной на всем отрезке $[a, b]$.

Из - за того, что у второй функции 4 производная улетает на бесконечность в точках 0 и 1, то отсюда следует, что ни о какой хорошей оценке нет и речи.

У третьей функции 4 производная тождественно ноль всюду, за исключением точки 0, в которой ее просто нет. Поэтому снова не получится сделать качественную оценку.

Разберемся с первой функцией (найдём погрешность). Так как 4 производная у $\sin(x)$ -- это снова $\sin(x)$, а также на отрезке $[0, \pi]$ у $\sin(x)$ максимум находится в точке $\pi/2$ и равен 1, тогда при $n=100$ получаем следующее:
$$ \mid I-I_{h}\mid \leq \frac{\pi^5}{2880\cdot100^4} = 1.062568e-09 $$

In [22]:
aprior = round((np.pi ** 5) / (2880 * 100 ** 4), 15)
print(aprior)

1.062568e-09


С другой стороны, используем апостериорную оценку Рунге
$$ I-I_h \thickapprox \frac{2^p(I_{h/2} - I_h)}{2^p-1}, $$
где $p=4$ в методе Симпсона.

Тогда получаем:

In [23]:
apost = round(2 ** 4 * (f_1_200 - num_1)/(2 ** 4 - 1), 15)
print(apost)

-6.76473e-10


В итоге получаем, что апостериорная оценка (по модулю) дала меньшую погрешность, чем априорная. Что и есть хороший знак.

In [24]:
abs(apost) < aprior

True

# Метод Гаусса

## Compute

**Вычислим все три интеграла на равномерной сетке со 100 интервалами методом Гаусса.**

In [25]:
num_4 = NI('gauss', 0, np.pi, 100)

In [26]:
print(f'Для первой функции: {num_4}')

Для первой функции: 2.000000099991001


In [27]:
num_5 = NI('gauss', 0, 10, 100, f_2)

In [28]:
print(f'Для второй функции: {num_5}') # Заметим, что все хорошо посчиталось, так получилось, что мы не попали в 0 и 1

Для второй функции: 23.960650868600275


In [29]:
num_6 = NI('gauss', -1, 2, 100, f_3)

In [30]:
print(f'Для третьей функции: {num_6}')

Для третьей функции: 2.500010440845493


## Априорная оценка

Априорная оценка для квадратур Гаусса при достаточно гладкой функции $f(x)$ на равномерной сетке с шагом $h=(b-a)/N$ при вычисление приближенного значения $I_h$ интеграла $I$ имеет вид:
$$ \mid I - I_h\mid < \frac{M_{2n}\cdot (b-a)\cdot h^{2n}}{(2n+1)\cdot(n!)^2}, $$
где $M_{2n}$ -- максимум $2n$ производной на отрезке $[a, b]$. **Обратим внимание, $n$ - количество узлов, а $N$ - интервалов!**

Покажем порядок на первой функции (с остальными уже поняли, что так не получится). У нас $n=3$, поэтому и порядок должен быть равен 6. Можно показать это с помощью апостериорной оценки Рунге, как мы это делали раньше.

In [32]:
g_1_2 = NI('gauss', 0, np.pi, 2)
g_1_4 = NI('gauss', 0, np.pi, 4)
g_1_8 = NI('gauss', 0, np.pi, 8)
print(f'Для 2 интервалов: {g_1_2}\nДля 40 интервалов: {g_1_4}\nДля 80 интервалов: {g_1_8}')

np.log2((g_1_4 - g_1_2)/(g_1_8 - g_1_4))

Для 2 интервалов: 2.0000163194296157
Для 40 интервалов: 2.000000332126719
Для 80 интервалов: 2.000000102246964


6.1199033442234345

Получили примерно 6, что соответствует теоретической оценке.